In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from requests_futures.sessions import FuturesSession
from urllib.parse import urljoin
from collections import namedtuple

In [5]:
from selenium import webdriver
driver = webdriver.Chrome(r'C:\Users\Admin\Desktop\chromedriver.exe')
driver.get("http://www.google.com")
el=driver.find_elements_by_xpath("//button[contains(string(), 'đầu tiên')]")[0]

action = webdriver.common.action_chains.ActionChains(driver)
action.move_to_element_with_offset(el, 5, 5)
action.click()
action.perform()

IndexError: list index out of range

In [36]:
TechDetailed = namedtuple('TechDetailed', 'title, company, description, requirement')
def detail(response):
    soup = bs(response.text, 'lxml')
    title = soup.find('div', attrs={'class':'top-job-info'}).select('h1')[0].text
    company = soup.find('div', attrs = {'class':'tit_company'}).text
    description = soup.find_all('div', attrs={'class':'content_fck'})[0].text
    requirement = soup.find_all('div', attrs={'class':'content_fck'})[1].text

    return TechDetailed(title, company, description, requirement)

In [67]:
SCROLL_PAUSE_TIME = 0.5
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-infobars")
driver = webdriver.Chrome(r'C:\Users\Admin\Desktop\chromedriver.exe')
driver.get('https://www.jobstreet.vn/j?q=data+scientist&l=&sp=homepage')
#keyword = driver.find_element_by_name('keyword2')
#keyword.clear()
#keyword.send_keys('data analyst')
#keyword.send_keys(Keys.RETURN)
  
# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

start = time.time()
urls = []
while True:
    soup = bs(driver.page_source, 'lxml')

    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    '''
    #load data
    jobs = soup.find_all('h3', attrs={'class':'job'})
  
    for item in jobs:
        urls.append(item.select('a')[0]['href'])
    
    session = FuturesSession(max_workers=20)
    futures = [session.get(url) for url in urls]
    '''
    # check the last page == 1 ?
    xt=driver.find_elements_by_link_text("Kế tiếp")
    if len(xt) == 0:
        break
    driver.find_element_by_link_text("Kế tiếp").click()

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    #if new_height == last_height:
     #   break
    last_height = new_height

TimeoutException: Message: timeout
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)


In [46]:
details = [detail(future.result()) for future in futures]
data_dict = {'url': urls, 'title': [detail.title for detail in details], 'company': [detail.company for detail in details], 'description':[detail.description for detail in details], 'requirement':[detail.requirement for detail in details] }

AttributeError: 'NoneType' object has no attribute 'select'

In [40]:
data_frame = pd.DataFrame(data_dict)
end = time.time()

In [44]:
data_frame.to_csv('CareerBuilder-DA.csv')

In [43]:
end - start

1868.9361221790314